In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn import ensemble

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA 
from sklearn.feature_selection import RFE
from datetime import datetime

%matplotlib inline

In [2]:
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

C:\Users\gyans\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [4]:
y2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421097 entries, 0 to 421096
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(85), object(26)
memory usage: 356.6+ MB


In [18]:
y2015.id.nunique()

421097

In [20]:
cat_cols = y2015.select_dtypes(include=['object'])
print('column \t \t  unique values')
print('-----------------------------')
for i in cat_cols:
    
    column = cat_cols[i]
    
    print(i +'\t \t' +str(column.nunique()))


column 	 	  unique values
-----------------------------
id	 	421097
term	 	2
int_rate	 	110
grade	 	7
sub_grade	 	35
emp_title	 	120812
emp_length	 	12
home_ownership	 	4
verification_status	 	3
issue_d	 	12
loan_status	 	7
pymnt_plan	 	1
url	 	421095
desc	 	34
purpose	 	14
title	 	27
zip_code	 	914
addr_state	 	49
earliest_cr_line	 	668
revol_util	 	1211
initial_list_status	 	2
last_pymnt_d	 	25
next_pymnt_d	 	4
last_credit_pull_d	 	26
application_type	 	2
verification_status_joint	 	3


In [3]:
# Drop the columns with over 30 unique values, 
y2015.drop(['sub_grade','emp_title','url','desc','zip_code','addr_state','earliest_cr_line','revol_util'],1, inplace=True)

In [4]:
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [5]:
y2015 = y2015[:-2]

In [6]:
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')

In [26]:
y2015.loan_status.unique()

array(['Current', 'Fully Paid', 'Charged Off', 'Late (31-120 days)',
       'In Grace Period', 'Default', 'Late (16-30 days)'], dtype=object)

In [7]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

# 10 fold cross val with all Features

In [46]:
start_time = datetime.now()
score = cross_val_score(rfc, X, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print(score.mean())

Duration: 0:04:26.214800
[ 0.97957778  0.9801477   0.98145378  0.98157251  0.97679886  0.97706008
  0.96202712  0.98064547  0.97981333  0.98047784]
0.977957447809


In [8]:
pd.set_option('display.max_columns', None)
print(y2015.shape)
print(X.shape)
y2015.head()

(421095, 103)
(421095, 202)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85,379.39,C,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,credit_card,Credit card refinancing,33.18,0.0,0.0,33.0,2.0,11.0,2.0,19108.0,19.0,w,13668.88,13668.88,4519.68,4519.68,2331.12,2188.56,0.0,0.0,0.0,Jan-2017,379.39,Jan-2017,Jan-2017,0.0,33.0,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,31329.0,1.0,2.0,1.0,2.0,6.0,12221.0,91.0,1.0,4.0,14982.0,65.0,284700.0,0.0,0.0,1.0,6.0,2848.0,263953.0,17.6,0.0,0.0,131.0,294.0,11.0,6.0,2.0,11.0,76.0,11.0,76.0,2.0,6.0,9.0,6.0,8.0,6.0,9.0,11.0,9.0,11.0,0.0,0.0,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49,298.58,A,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,credit_card,Credit card refinancing,22.44,0.0,0.0,NaN,NaN,7.0,0.0,7722.0,9.0,w,6635.69,6635.69,3572.97,3572.97,2964.31,608.66,0.0,0.0,0.0,Jan-2017,298.58,Jan-2017,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,55387.0,0.0,3.0,0.0,0.0,47.0,47665.0,43.0,2.0,2.0,4744.0,50.0,13000.0,0.0,0.0,2.0,2.0,7912.0,4778.0,61.8,0.0,0.0,234.0,91.0,9.0,9.0,0.0,11.0,NaN,9.0,NaN,0.0,3.0,3.0,3.0,3.0,5.0,4.0,4.0,3.0,7.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49,777.55,A,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,debt_consolidation,Debt consolidation,26.02,0.0,1.0,NaN,NaN,9.0,0.0,20862.0,19.0,w,0.00,0.00,26224.23,26224.23,25000.00,1224.23,0.0,0.0,0.0,Sep-2016,20807.39,NaN,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,305781.0,0.0,3.0,0.0,1.0,13.0,47194.0,58.0,0.0,1.0,8937.0,57.0,38400.0,1.0,0.0,1.0,2.0,33976.0,17538.0,54.3,0.0,0.0,142.0,168.0,13.0,13.0,3.0,13.0,NaN,0.0,NaN,0.0,3.0,3.0,5.0,6.0,7.0,5.0,9.0,3.0,9.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49,858.05,A,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,debt_consolidation,Debt consolidation,21.60,0.0,0.0,42.0,NaN,16.0,0.0,51507.0,24.0,w,19263.77,19263.77,10271.36,10271.36,8736.23,1535.13,0.0,0.0,0.0,Jan-2017,858.05,Jan-2017,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,221110.0,0.0,2.0,0.0,0.0,27.0,23413.0,55.0,0.0,1.0,8752.0,61.0,79900.0,0.0,1.0,0.0,1.0,13819.0,16623.0,59.9,0.0,0.0,82.0,379.0,19.0,19.0,2.0,19.0,48.0,NaN,42.0,0.0,7.0,9.0,9.0,11.0,4.0,13.0,18.0,9.0,16.0,0.0,0.0,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8

In [19]:
filter_col = X.columns[X.columns.str.contains('pymnt')].tolist()
filter_col

['total_pymnt',
 'total_pymnt_inv',
 'last_pymnt_amnt',
 'pymnt_plan_n',
 'last_pymnt_d_Apr-2015',
 'last_pymnt_d_Apr-2016',
 'last_pymnt_d_Aug-2015',
 'last_pymnt_d_Aug-2016',
 'last_pymnt_d_Dec-2015',
 'last_pymnt_d_Dec-2016',
 'last_pymnt_d_Feb-2015',
 'last_pymnt_d_Feb-2016',
 'last_pymnt_d_Jan-2015',
 'last_pymnt_d_Jan-2016',
 'last_pymnt_d_Jan-2017',
 'last_pymnt_d_Jul-2015',
 'last_pymnt_d_Jul-2016',
 'last_pymnt_d_Jun-2015',
 'last_pymnt_d_Jun-2016',
 'last_pymnt_d_Mar-2015',
 'last_pymnt_d_Mar-2016',
 'last_pymnt_d_May-2015',
 'last_pymnt_d_May-2016',
 'last_pymnt_d_Nov-2015',
 'last_pymnt_d_Nov-2016',
 'last_pymnt_d_Oct-2015',
 'last_pymnt_d_Oct-2016',
 'last_pymnt_d_Sep-2015',
 'last_pymnt_d_Sep-2016',
 'next_pymnt_d_Feb-2017',
 'next_pymnt_d_Jan-2017',
 'next_pymnt_d_Jul-2016',
 'next_pymnt_d_Mar-2017']

In [20]:
x = X.drop(filter_col, 1)
           #['last_pymnt_amnt', 'out_prncp', 'last_pymnt_d_Jan-2017', 'out_prncp_inv']

In [23]:
print(X.shape)
print(x.shape)

(421095, 202)
(421095, 169)


In [24]:
filter_col = X.columns[X.columns.str.contains('prncp')].tolist()
x = X.drop(filter_col, 1)
print(X.shape)
print(x.shape)

(421095, 202)
(421095, 199)


# Removing the columns related to payment amount or outstanding principal

In [27]:
print('Removing the columns related to payment amount or outstanding principal \n ')
start_time = datetime.now()
score = cross_val_score(rfc, x, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print(score.mean())

Removing the columns related to payment amount or outstanding principal 
 
Duration: 0:04:37.748357
[ 0.95305265  0.97791551  0.9794353   0.98078887  0.96703871  0.96965091
  0.94490489  0.97879313  0.97511103  0.97955161]
0.970624260378


In [28]:
df = pd.DataFrame()
df['imp']= rfc.fit(X, Y).feature_importances_
df['features'] = X.columns
sorted_features = df.sort_values(by='imp', ascending=False)

# Selecting only the top 50 features based on feature importance

In [29]:
print('Selecting only the top 50 features based on feature importance \n')
top_50_features = np.asarray(sorted_features['features'].iloc[:50])
x2 =X[top_50_features]
start_time = datetime.now()
score = cross_val_score(rfc, x2, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print(score.mean())

Selecting only the top 50 features based on feature importance 

Duration: 0:04:50.555411
[ 0.97957778  0.98043267  0.98195246  0.98185747  0.98078841  0.9809309
  0.97926809  0.98066922  0.98007457  0.9798841 ]
0.980543566557


# Selecting only the top 25 features based on feature importance

In [51]:
print('Selecting only the top 25 features based on feature importance \n')
top_25_features = np.asarray(sorted_features['features'].iloc[:25])
x3 =X[top_25_features]
start_time = datetime.now()
score = cross_val_score(rfc, x3, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print(score.mean())

Selecting only the top 25 features based on feature importance 

Duration: 0:02:59.915287
[ 0.9731899   0.97736933  0.97910285  0.98138254  0.97010211  0.97573023
  0.97387732  0.97893562  0.97449355  0.98000285]
0.976418630397


# Selecting only the top 10 features based on feature importance

In [52]:
print('Selecting only the top 10 features based on feature importance \n')
top_10_features = np.asarray(sorted_features['features'].iloc[:10])
x4 =X[top_10_features]
start_time = datetime.now()
score = cross_val_score(rfc, x4, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print(score.mean())

Selecting only the top 10 features based on feature importance 

Duration: 0:02:44.146842
[ 0.9603429   0.96863052  0.97485218  0.97734559  0.96689622  0.97219188
  0.9689615   0.97570591  0.96858005  0.97843538]
0.971194212724


# Selecting only the top 5 features based on feature importance

In [53]:
print('Selecting only the top 5 features based on feature importance \n')
top_5_features = np.asarray(sorted_features['features'].iloc[:5])
x5 =X[top_5_features]
start_time = datetime.now()
score = cross_val_score(rfc, x5, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print(score.mean())

Selecting only the top 5 features based on feature importance 

Duration: 0:02:19.277681
[ 0.92807105  0.92341668  0.92835601  0.93775973  0.91056756  0.93248635
  0.92956375  0.93697309  0.91421854  0.94110103]
0.928251379703


# Selecting only the top 2 features based on feature importance

In [54]:
print('Selecting only the top 2 features based on feature importance \n')
top_2_features = np.asarray(sorted_features['features'].iloc[:2])
x6 =X[top_2_features]
start_time = datetime.now()
score = cross_val_score(rfc, x6, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print(score.mean())

Selecting only the top 2 features based on feature importance 

Duration: 0:02:04.373870
[ 0.86839543  0.93441144  0.91092589  0.94262782  0.94039421  0.9337687
  0.91279774  0.92543162  0.93366899  0.9397948 ]
0.924221662444


In [74]:
sorted_features['features'].iloc[:2]

16      out_prncp_inv
24    last_pymnt_amnt
Name: features, dtype: object

# feature selection using the threshold method

In [30]:
from sklearn.feature_selection import VarianceThreshold
#(.8 * (1 - .8))
print('Using Threshold of 0.1 \n ')
start_time = datetime.now()
sel = VarianceThreshold(threshold=0.1)
X_thr = sel.fit_transform(X)

score = cross_val_score(rfc, X_thr, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print('Average Score : ',score.mean())

Using Threshold of 0.1 
 
Duration: 0:07:33.010807
[ 0.9636912   0.96782313  0.96421363  0.96373869  0.96160057  0.96309665
  0.96209836  0.96188463  0.95815423  0.96067069]
Average Score :  0.962697178014


In [31]:
print(X.shape)
print(X_thr.shape)

(421095, 202)
(421095, 75)


In [32]:
selector = VarianceThreshold(0.1) 
#Fit the Model
selector.fit(X)
df2 = X.loc[:,selector.get_support(indices=False)] 
print(df2.shape)
print(df2.columns.values)


(421095, 75)
['id' 'member_id' 'loan_amnt' 'funded_amnt' 'funded_amnt_inv' 'int_rate'
 'installment' 'annual_inc' 'dti' 'delinq_2yrs' 'inq_last_6mths' 'open_acc'
 'pub_rec' 'revol_bal' 'total_acc' 'out_prncp' 'out_prncp_inv'
 'total_pymnt' 'total_pymnt_inv' 'total_rec_prncp' 'total_rec_int'
 'total_rec_late_fee' 'recoveries' 'collection_recovery_fee'
 'last_pymnt_amnt' 'tot_coll_amt' 'tot_cur_bal' 'total_rev_hi_lim'
 'acc_open_past_24mths' 'avg_cur_bal' 'delinq_amnt' 'mo_sin_old_rev_tl_op'
 'mo_sin_rcnt_rev_tl_op' 'mo_sin_rcnt_tl' 'mort_acc'
 'num_accts_ever_120_pd' 'num_actv_bc_tl' 'num_actv_rev_tl' 'num_bc_sats'
 'num_bc_tl' 'num_il_tl' 'num_op_rev_tl' 'num_rev_tl_bal_gt_0' 'num_sats'
 'num_tl_90g_dpd_24m' 'num_tl_op_past_12m' 'pct_tl_nvr_dlq'
 'pub_rec_bankruptcies' 'tax_liens' 'tot_hi_cred_lim' 'total_bal_ex_mort'
 'total_bc_limit' 'total_il_high_credit_limit' 'term_ 36 months'
 'term_ 60 months' 'grade_A' 'grade_B' 'grade_C' 'grade_D'
 'emp_length_10+ years' 'home_ownership_MORTGA

# Feature selection using PCA

# PCA for 5 features

In [33]:
start_time = datetime.now()
X_std = StandardScaler().fit_transform(X)
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_std)

score = cross_val_score(rfc, X_pca, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print('Accuracy : ', score.mean())

Duration: 0:04:22.933356
[ 0.86400228  0.87646933  0.88905512  0.89646411  0.89964379  0.90132985
  0.90358356  0.89503432  0.87358396  0.81494324]
Accuracy :  0.881410955003


# PCA for 10 features

In [34]:
start_time = datetime.now()
X_std = StandardScaler().fit_transform(X)
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_std)

score = cross_val_score(rfc, X_pca, Y, cv=10)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
print(score)
print('Accuracy : ',score.mean())

Duration: 0:05:56.814046
[ 0.91971219  0.93194177  0.93450642  0.93678611  0.93357872  0.93424365
  0.93557197  0.93276972  0.93604389  0.93088871]
Accuracy :  0.932604315084


When we did PCA with 5 features the accuracy dropped below 90%. The PCA for 10 components was 93%. When Selected only 2 features based on feature importance the accuracy was 92%.